<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/factor_investing/ghz_sorts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import urllib.parse
from sqlalchemy import create_engine

chars = [
    "bm",
    "ep",
    "cashpr",
    "dy",
    "lev",
    "sp",
    "roic",
    "rd_sale",
    "rd_mve",
    "agr",
    "gma",
    "chcsho",
    "lgr",
    "acc",
    "pctacc",
    "cfp",
    "absacc",
    "age",
    "chinv",
    "hire",
    "sgr",
    "pchsale_pchinvt",
    "pchsale_pchrect",
    "pchgm_pchsale",
    "pchsale_pchxsga",
    "depr",
    "pchdepr",
    "invest",
    "egr",
    "grcapx",
    "tang",
    "sin",
    "currat",
    "pchcurrat",
    "quick",
    "pchquick",
    "salecash",
    "salerec",
    "saleinv",
    "pchsaleinv",
    "cashdebt",
    "realestate",
    "divi",
    "divo",
    "securedind",
    "secured",
    "convind",
    "grltnoa",
    "rd",
    "operprof",
    "ps",
    "chpmia",
    "chatoia",
    "chempia",
    "bm_ia",
    "pchcapx_ia",
    "tb",
    "cfp_ia",
    "mve_ia",
    "herf",
    "orgcap",
    "mve",
    "chtx",
    "roaq",
    "roeq",
    "rsup",
    "stdacc",
    "roavol",
    "stdcf",
    "cash",
    "cinvest",
    "nincr",
    "sue",
    "aeavol",
    "ear",
    "ms",
    "disp",
    "chfeps",
    "fgr5yr",
    "nanalyst",
    "sfe",
    "chnanalyst",
    "mom6m",
    "mom12m",
    "mom36m",
    "mom1m",
    "dolvol",
    "chmom",
    "turn",
    "ipo",
    "indmom",
    "maxret",
    "retvol",
    "baspread",
    "std_dolvol",
    "std_turn",
    "ill",
    "zerotrade",
    "beta",
    "betasq",
    "pricedelay",
    "idiovol",
]

intchars = [
    "age",
    "sin",
    "divi",
    "divo",
    "securedind",
    "convind",
    "rd",
    "ps",
    "nincr",
    "ms",
    "ipo",
]

chars = [x for x in chars if x not in intchars]
chars = np.sort(chars)
labels = ["Lo 20", "Qnt 2", "Qnt 3", "Qnt 4", "Hi 20"]

server = "eu-az-sql-serv1.database.windows.net:1433"
database = "dgn022k6348dcyh"
username = "uhgrque4d8p77hf"
password = "FfWrgFcK$Vnk@9BAgKH4nbEDF"
password = urllib.parse.quote_plus(password)

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database
conn = create_engine(string).connect()


char = 'age'        # example characteristics

df = pd.read_sql(
    " select date, ret, " + char + " from ghz where date>='2000-01-01' ", conn
)
df = df.dropna()

def cut(x):
    try:
        out = pd.qcut(x, 5, labels=labels)
    except:
        out = pd.Series(np.nan, index=x.index)
    return out

df["quintile"] = df.groupby("date")[char].apply(cut)

df = df.dropna(subset=["quintile"])
rets = df.groupby(["date", "quintile"]).ret.mean().unstack()

rets

quintile,Hi 20,Lo 20,Qnt 2,Qnt 3,Qnt 4
date,,,,,
2000-01-31,-0.058879,-0.006268,-0.022532,-0.022202,-0.037820
2000-02-29,-0.012514,0.202780,0.155891,0.154869,-0.001974
2000-03-31,0.075751,-0.028457,-0.011562,0.026104,0.088343
2000-04-28,-0.003257,-0.142018,-0.083510,-0.058083,0.002343
2000-05-31,-0.010654,-0.076697,-0.061397,-0.029640,-0.000577
...,...,...,...,...,...
2021-08-31,0.010540,0.012788,0.027826,0.021445,0.017105
2021-09-30,-0.069383,-0.021897,-0.030265,-0.021262,-0.033462
2021-10-29,0.066600,0.053622,0.044204,0.040113,0.043888
